In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import nanogpt_train

torch.manual_seed(15)

### Tokenize the data

In [ ]:
from tokenize import Tokenization_Procedure

#byte
processing_type = "byte"
tokenizing_process = Tokenization_Procedure(processing_type = processing_type)

#bigram
#tokenizing_process = Tokenization_Procedure(processing_type = 'bigram')

In [2]:
#Get the input text
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    
#What is the length of the dataset in characters
#print(f"length of the dataset in characters: {len(text)}")


##Sort the text
if p
data = tokenizing_process.encode(text)

data = torch.tensor(encode(text), dtype = torch.long)
#print("Every integer in data represents a single character ", data.shape)


#Train and Test Splits
n = int(0.9*len(data))

train_data = data[:n]
val_data = data[n:]



All Chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Total number: 65 chars


In [3]:
#hyperparameters section
batch_size = 64 #How many independent sequences of characters will we process in parallel
context_length = 256 #-Cody's version of block size. What is the maximum context length for predictions
max_iters = 3500#3000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
gen_dropout = 0.2
num_blocks = 2
num_heads = 6
attn_dropout = 0.2
residual_cxn_dropout = 0.2
head_size = n_embd//num_heads
        



In [4]:
model_args = {'batch_size':64, 
              'context_length':256,
              'max_iters':3500, 
              'eval_interval':500,
              'learning_rate':3e-4, 
              'device':device, 
              'eval_iters':eval_iters,
              'n_embd':n_embd,
              'gen_dropout': gen_dropout,
              'num_blocks':num_blocks,
              'num_heads': num_heads,
              'head_size':head_size,
              'attn_dropout': attn_dropout, 
              'residual_cxn_dropout':residual_cxn_dropout,
              'vocab_size':vocab_size,
              'head_size':head_size}


In [5]:
##Run the model
#Initiate the model
gptconfig = nanogpt_train.GPTConfig(**model_args)
gptconfig.head_size

64

In [6]:
##Run the model
model = nanogpt_train.PrelimGPT(gptconfig)
print(model)

PrelimGPT(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(256, 384)
  (attention_blocks): Sequential(
    (0): Block(
      (multihead_attention): CausalSelfAttention(
        (kqv): Linear(in_features=384, out_features=1152, bias=False)
        (dropout): Dropout(p=0.2, inplace=False)
        (attn_dropout): Dropout(p=0.2, inplace=False)
        (residual_dropout): Dropout(p=0.2, inplace=False)
        (projection): Linear(in_features=384, out_features=384, bias=True)
      )
      (ln1): LayerNorm()
      (MLP): MLP(
        (sequential_MLP): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=False)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1536, out_features=384, bias=False)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln2): LayerNorm()
    )
    (1): Block(
      (multihead_attention): CausalSelfAttention(
        (kqv): Linear(in_features=384, out_features=1152, 

In [7]:
#Batch splitting
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,)) #tensor of four integers representing four characters from the text
    # So for the random characters, we get i and then up to the block size. Then we put in a bigger tensor
    x = torch.stack([data[i:i+context_length] for i in ix])
    #same thing for y
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    
    #for gpu compatibility
    x = x.to(device)
    y= y.to(device)
    
    return x,y

@torch.no_grad()#Everything that happens inside this function, don't keep track of grad on estimate loss
def estimate_loss(model, split):
    
    """This function is used for switching between eval and train model to routinely
    evaluate model performance"""
    
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    
    #Switch back to eval 
    model.train()
    return out


In [ ]:
##Run the model
#Initiate the model
gptconfig = nanogpt_train.GPTConfig(**model_args)
model = nanogpt_train.PrelimGPT(gptconfig)

print(model)

m = model.to(device)

#create a PyTorch optimizer
optimizer = torch.optim.AdamW(params = m.parameters(), lr = learning_rate)


for _ in range(max_iters):

    #evaluate loss
    if _ % eval_interval == 0:
        losses = estimate_loss(model, 'eval')
        print(f"step {_}: train loss {losses['train']:4f}, val loss {losses['val']:.4f}")
    
    #Get the batches
    xb, yb = get_batch('train')
    
    #Run the forward pass
    logits, loss = m(xb, yb)
    
    #zero the gradients prior to running gradient calculations
    optimizer.zero_grad(set_to_none=True)
    
    #Calculate the gradients and use them to update the parameters
    loss.backward()
    
    #step the optimizer to update the parameters
    optimizer.step()
        
    
#Generation section
#Turn it into a list and decode
#batch_of_interest = [str(x) for x in batch_of_interest]
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(''.join(decode(m.generate(context, max_new_tokens = 500)[0].tolist())))

PrelimGPT(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(256, 384)
  (attention_blocks): Sequential(
    (0): Block(
      (multihead_attention): CausalSelfAttention(
        (kqv): Linear(in_features=384, out_features=1152, bias=False)
        (dropout): Dropout(p=0.2, inplace=False)
        (attn_dropout): Dropout(p=0.2, inplace=False)
        (residual_dropout): Dropout(p=0.2, inplace=False)
        (projection): Linear(in_features=384, out_features=384, bias=True)
      )
      (ln1): LayerNorm()
      (MLP): MLP(
        (sequential_MLP): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=False)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1536, out_features=384, bias=False)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln2): LayerNorm()
    )
    (1): Block(
      (multihead_attention): CausalSelfAttention(
        (kqv): Linear(in_features=384, out_features=1152, 

step 0: train loss 4.559170, val loss 4.5565
step 500: train loss 2.437334, val loss 2.4772
step 1000: train loss 2.050138, val loss 2.1246
step 1500: train loss 1.862802, val loss 1.9863
step 2000: train loss 1.709359, val loss 1.8696
step 2500: train loss 1.651126, val loss 1.8345
step 3000: train loss 1.620306, val loss 1.8073


THERMIONE:
Nor Bolicy inde int, womb and sun, sta bugling own ren,
Of mindiolack hou and led-vilb.
Of here am slive! restiand rer to speak you stant shal shall,fectinsad
Advour frous nature.

GENUMERS:
And their rough !
Why, confothen aution
spear dires, knot livit EDWAREDUCK:
NGwe Come cargivire, my wousanot theed's an the.
My dge is sall Ricere,
To my his hom ingee
Anualll ar comee ther, fompot wome yed: I'll t wir heell now shalencoms hiny sowanot my marr,
Rome dow yone nothelf.

DUCAEnguere
